<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/03_Analisis_Fase_2C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
================================================================================
FASE 2C - PCA, CLUSTERING Y ANALISIS DE REDUNDANCIA DE METRICAS
================================================================================
Trabajo Fin de Master: Evaluacion Comparativa de Tecnicas de Segmentacion
en Fotografia de Retrato

Autor: Jesus L.
Universidad: Universitat Oberta de Catalunya (UOC)
Fecha: Diciembre 2025

DESCRIPCION:
Fase 2C del analisis comparativo. Realiza analisis exploratorio multivariante
sobre las metricas de segmentacion calculadas en Fase 2A, incluyendo:
- Reduccion de dimensionalidad mediante PCA
- Clustering de fotografias por nivel de dificultad
- Identificacion de metricas redundantes
- Visualizaciones para el Capitulo 4 del TFM

DEPENDENCIAS DE DATOS:
- Requiere Fase 2B completada (metricas_fusionadas.csv)
- Utiliza metricas geometricas Shapely y metricas clasicas

ANALISIS INCLUIDOS:
1. PCA de metricas de segmentacion (geometricas + clasicas)
2. Clustering K-means de fotografias por dificultad (k=3)
3. Clustering jerarquico con dendrograma
4. Matriz de correlaciones entre metricas
5. Identificacion de metricas redundantes (|r| > 0.9)
6. Visualizaciones: biplot, heatmap, scatter PCA

ESTRUCTURA DE SALIDA:
/TFM/3_Analisis/fase2c_pca_clustering/
├── pca_componentes.csv           # Scores PC1, PC2, PC3 por evaluacion
├── pca_varianza_explicada.csv    # Varianza explicada por componente
├── pca_loadings.csv              # Pesos de cada metrica en cada PC
├── clusters_fotografias.csv      # Asignacion de cluster por fotografia
├── clusters_centroides.csv       # Centroides de cada cluster
├── correlaciones_metricas.csv    # Matriz de correlaciones
├── metricas_redundantes.csv      # Pares con |r| > 0.9
├── visualizaciones/
│   ├── pca_varianza_explicada.png
│   ├── pca_biplot.png
│   ├── pca_scatter_clusters.png
│   ├── dendrograma_fotografias.png
│   ├── heatmap_correlaciones.png
│   └── clusters_caracterizacion.png


================================================================================
"""

In [ ]:
# =============================================================================
# IMPORTS
# =============================================================================

import json
import logging
import sys
import warnings
from dataclasses import dataclass, field
from datetime import datetime
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple

import numpy as np
import pandas as pd
from scipy import stats
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import pdist

# Sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score

# Visualizacion
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

# Suprimir warnings de convergencia
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

In [ ]:
# =============================================================================
# CONFIGURACION DE VISUALIZACION
# =============================================================================

plt.rcParams.update({
    'figure.figsize': (12, 8),
    'figure.dpi': 150,
    'savefig.dpi': 300,
    'font.size': 10,
    'axes.titlesize': 12,
    'axes.labelsize': 10,
    'xtick.labelsize': 9,
    'ytick.labelsize': 9,
    'legend.fontsize': 9,
    'figure.titlesize': 14,
    'axes.grid': True,
    'grid.alpha': 0.3,
    'axes.spines.top': False,
    'axes.spines.right': False
})

sns.set_style("whitegrid")

# Paleta de colores para modelos
COLORES_MODELOS = {
    'yolov8': '#2ecc71',      # Verde
    'oneformer': '#3498db',   # Azul
    'sam2': '#e74c3c',        # Rojo
    'sam2_prompts': '#9b59b6', # Purpura
    'mask2former': '#f39c12', # Naranja
    'bodypix': '#1abc9c'      # Turquesa
}

# Paleta para clusters
COLORES_CLUSTERS = {
    0: '#27ae60',  # Verde - Facil
    1: '#f1c40f',  # Amarillo - Medio
    2: '#e74c3c'   # Rojo - Dificil
}

NOMBRES_CLUSTERS = {
    0: 'Facil',
    1: 'Medio',
    2: 'Dificil'
}

In [ ]:
# =============================================================================
# CONFIGURACION
# =============================================================================

@dataclass
class ConfiguracionFase2C:
    """
    Configuracion central para el pipeline de Fase 2C.

    Attributes:
        ruta_base_tfm: Ruta raiz del proyecto TFM
        ruta_datos_fase2b: Ruta al CSV fusionado de Fase 2B
        ruta_salida: Directorio para resultados de Fase 2C
        n_componentes_pca: Numero de componentes PCA a retener
        n_clusters: Numero de clusters para K-means
        umbral_redundancia: Umbral de correlacion para considerar redundancia
        random_state: Semilla para reproducibilidad
    """
    ruta_base_tfm: Path
    ruta_datos_fase2b: Path = None
    ruta_salida: Path = None
    n_componentes_pca: int = 5
    n_clusters: int = 3
    umbral_redundancia: float = 0.90
    random_state: int = 42

    def __post_init__(self):
        """Inicializa rutas derivadas si no se especifican."""
        if self.ruta_datos_fase2b is None:
            self.ruta_datos_fase2b = (
                self.ruta_base_tfm / "3_Analisis" / "fase2b_correlaciones" /
                "metricas_fusionadas.csv"
            )
        if self.ruta_salida is None:
            self.ruta_salida = self.ruta_base_tfm / "3_Analisis" / "fase2c_pca_clustering"

In [ ]:
# =============================================================================
# CONSTANTES: METRICAS PARA ANALISIS
# =============================================================================

# Metricas de segmentacion para PCA
METRICAS_SEGMENTACION = [
    # Metricas clasicas
    'iou',
    'dice',
    'precision',
    'recall',
    'boundary_iou',

    # Metricas geometricas Shapely - Basicas
    'pred_area',
    'pred_perimeter',
    'pred_compactness',
    'pred_convexity',
    'pred_solidity',

    # Metricas geometricas - Bounding box
    'pred_bbox_aspect_ratio',
    'pred_extent',

    # Metricas geometricas - Centroide
    'pred_centroid_x_norm',
    'pred_centroid_y_norm',

    # Metricas de comparacion con GT
    'area_ratio',
    'perimeter_ratio',
    'centroid_distance',
    'hausdorff_distance',
]

# Metricas alternativas si las anteriores no existen
METRICAS_ALTERNATIVAS = {
    'pred_area': ['area_pred', 'mask_area'],
    'pred_perimeter': ['perimeter_pred', 'mask_perimeter'],
    'pred_compactness': ['compactness', 'compacidad'],
    'pred_convexity': ['convexity', 'convexidad'],
    'pred_solidity': ['solidity', 'solidez'],
    'boundary_iou': ['biou', 'boundary_score'],
    'hausdorff_distance': ['hausdorff', 'hd'],
}


# =============================================================================
# CLASE: CARGADOR DE DATOS
# =============================================================================

class CargadorDatosFase2C:
    """
    Gestiona la carga y preparacion de datos para Fase 2C.
    """

    def __init__(self, config: ConfiguracionFase2C, logger: logging.Logger):
        """
        Inicializa el cargador.

        Args:
            config: Configuracion de Fase 2C
            logger: Logger para registro
        """
        self.config = config
        self.logger = logger

    def cargar_datos_fusionados(self) -> pd.DataFrame:
        """
        Carga el dataset fusionado de Fase 2B.

        Returns:
            DataFrame con metricas fusionadas
        """
        self.logger.info(f"Cargando datos de: {self.config.ruta_datos_fase2b}")

        if not self.config.ruta_datos_fase2b.exists():
            raise FileNotFoundError(
                f"No se encuentra el archivo: {self.config.ruta_datos_fase2b}\n"
                "Ejecute Fase 2B primero."
            )

        df = pd.read_csv(self.config.ruta_datos_fase2b)

        self.logger.info(f"  Filas: {len(df)}")
        self.logger.info(f"  Columnas: {len(df.columns)}")
        self.logger.info(f"  Modelos: {df['modelo'].nunique()}")
        self.logger.info(f"  Fotografias: {df['codigo_foto'].nunique()}")

        return df

    def identificar_metricas_disponibles(self, df: pd.DataFrame) -> List[str]:
        """
        Identifica que metricas de segmentacion estan disponibles.

        Args:
            df: DataFrame con datos

        Returns:
            Lista de nombres de metricas disponibles
        """
        metricas_disponibles = []

        for metrica in METRICAS_SEGMENTACION:
            if metrica in df.columns:
                # Verificar que tiene valores no nulos
                if df[metrica].notna().sum() > 0:
                    metricas_disponibles.append(metrica)
            else:
                # Buscar alternativas
                if metrica in METRICAS_ALTERNATIVAS:
                    for alt in METRICAS_ALTERNATIVAS[metrica]:
                        if alt in df.columns and df[alt].notna().sum() > 0:
                            metricas_disponibles.append(alt)
                            self.logger.info(f"  Usando '{alt}' en lugar de '{metrica}'")
                            break

        self.logger.info(f"Metricas disponibles para PCA: {len(metricas_disponibles)}")

        return metricas_disponibles

    def preparar_matriz_metricas(self, df: pd.DataFrame,
                                  metricas: List[str]) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """
        Prepara la matriz de metricas para PCA, manejando valores faltantes.

        Args:
            df: DataFrame original
            metricas: Lista de metricas a incluir

        Returns:
            Tupla (DataFrame con metricas, DataFrame con metadata)
        """
        self.logger.info("Preparando matriz de metricas para PCA...")

        # Extraer columnas de metricas
        df_metricas = df[metricas].copy()

        # Extraer metadata (identificadores)
        cols_meta = ['modelo', 'config_codigo', 'codigo_foto']
        cols_meta_disponibles = [c for c in cols_meta if c in df.columns]
        df_meta = df[cols_meta_disponibles].copy()

        # Reportar valores faltantes
        nulos_por_metrica = df_metricas.isnull().sum()
        if nulos_por_metrica.sum() > 0:
            self.logger.warning("Valores faltantes detectados:")
            for col, nulos in nulos_por_metrica[nulos_por_metrica > 0].items():
                self.logger.warning(f"  {col}: {nulos} ({100*nulos/len(df):.1f}%)")

        # Eliminar filas con demasiados nulos (>50% de metricas)
        umbral_nulos = len(metricas) * 0.5
        filas_validas = df_metricas.isnull().sum(axis=1) < umbral_nulos

        n_eliminadas = (~filas_validas).sum()
        if n_eliminadas > 0:
            self.logger.warning(f"Eliminando {n_eliminadas} filas con >50% valores faltantes")

        df_metricas = df_metricas[filas_validas].copy()
        df_meta = df_meta[filas_validas].copy()

        # Imputar valores faltantes restantes con la mediana
        for col in df_metricas.columns:
            if df_metricas[col].isnull().any():
                mediana = df_metricas[col].median()
                df_metricas[col].fillna(mediana, inplace=True)

        self.logger.info(f"Matriz final: {df_metricas.shape[0]} filas x {df_metricas.shape[1]} metricas")

        return df_metricas, df_meta


# =============================================================================
# CLASE: ANALIZADOR PCA
# =============================================================================

class AnalizadorPCA:
    """
    Ejecuta analisis de componentes principales sobre metricas de segmentacion.
    """

    def __init__(self, config: ConfiguracionFase2C, logger: logging.Logger):
        """
        Inicializa el analizador PCA.

        Args:
            config: Configuracion de Fase 2C
            logger: Logger para registro
        """
        self.config = config
        self.logger = logger

        self.scaler = StandardScaler()
        self.pca = None
        self.X_scaled = None
        self.X_pca = None
        self.feature_names = None

    def ejecutar_pca(self, df_metricas: pd.DataFrame) -> Dict[str, Any]:
        """
        Ejecuta PCA sobre la matriz de metricas.

        Args:
            df_metricas: DataFrame con metricas (filas=evaluaciones, cols=metricas)

        Returns:
            Diccionario con resultados del PCA
        """
        self.logger.info("Ejecutando PCA...")

        self.feature_names = df_metricas.columns.tolist()

        # Estandarizar datos
        self.X_scaled = self.scaler.fit_transform(df_metricas)

        # Ejecutar PCA
        n_components = min(self.config.n_componentes_pca, len(self.feature_names))
        self.pca = PCA(n_components=n_components, random_state=self.config.random_state)
        self.X_pca = self.pca.fit_transform(self.X_scaled)

        # Calcular varianza explicada
        varianza_explicada = self.pca.explained_variance_ratio_
        varianza_acumulada = np.cumsum(varianza_explicada)

        self.logger.info(f"  Componentes retenidos: {n_components}")
        self.logger.info(f"  Varianza explicada total: {varianza_acumulada[-1]*100:.1f}%")

        for i, (var, acum) in enumerate(zip(varianza_explicada, varianza_acumulada)):
            self.logger.info(f"    PC{i+1}: {var*100:.1f}% (acumulada: {acum*100:.1f}%)")

        # Preparar resultados
        resultados = {
            'n_componentes': n_components,
            'varianza_explicada': varianza_explicada.tolist(),
            'varianza_acumulada': varianza_acumulada.tolist(),
            'loadings': self.pca.components_.tolist(),
            'feature_names': self.feature_names,
            'scores': self.X_pca.tolist()
        }

        return resultados

    def obtener_dataframe_scores(self, df_meta: pd.DataFrame) -> pd.DataFrame:
        """
        Genera DataFrame con scores PCA y metadata.

        Args:
            df_meta: DataFrame con metadata (modelo, config, foto)

        Returns:
            DataFrame con PC1, PC2, ... y metadata
        """
        # Crear DataFrame de scores
        cols_pc = [f'PC{i+1}' for i in range(self.X_pca.shape[1])]
        df_scores = pd.DataFrame(self.X_pca, columns=cols_pc)

        # Combinar con metadata
        df_scores = pd.concat([df_meta.reset_index(drop=True), df_scores], axis=1)

        return df_scores

    def obtener_dataframe_loadings(self) -> pd.DataFrame:
        """
        Genera DataFrame con loadings (pesos de cada variable en cada PC).

        Returns:
            DataFrame con loadings
        """
        cols_pc = [f'PC{i+1}' for i in range(self.pca.n_components_)]

        df_loadings = pd.DataFrame(
            self.pca.components_.T,
            index=self.feature_names,
            columns=cols_pc
        )
        df_loadings.index.name = 'metrica'

        return df_loadings

    def obtener_dataframe_varianza(self) -> pd.DataFrame:
        """
        Genera DataFrame con varianza explicada por componente.

        Returns:
            DataFrame con varianza explicada
        """
        n_comp = self.pca.n_components_

        df_varianza = pd.DataFrame({
            'componente': [f'PC{i+1}' for i in range(n_comp)],
            'varianza_explicada': self.pca.explained_variance_ratio_,
            'varianza_acumulada': np.cumsum(self.pca.explained_variance_ratio_),
            'eigenvalue': self.pca.explained_variance_
        })

        return df_varianza

    def identificar_variables_importantes(self, n_top: int = 5) -> Dict[str, List[Tuple[str, float]]]:
        """
        Identifica las variables mas importantes para cada componente.

        Args:
            n_top: Numero de variables top a retornar por componente

        Returns:
            Diccionario {componente: [(variable, loading), ...]}
        """
        resultado = {}

        for i in range(self.pca.n_components_):
            loadings = self.pca.components_[i]

            # Ordenar por valor absoluto
            indices_ordenados = np.argsort(np.abs(loadings))[::-1]

            top_vars = [
                (self.feature_names[idx], float(loadings[idx]))
                for idx in indices_ordenados[:n_top]
            ]

            resultado[f'PC{i+1}'] = top_vars

        return resultado


# =============================================================================
# CLASE: ANALIZADOR CLUSTERING
# =============================================================================

class AnalizadorClustering:
    """
    Ejecuta clustering de fotografias por nivel de dificultad de segmentacion.
    """

    def __init__(self, config: ConfiguracionFase2C, logger: logging.Logger):
        """
        Inicializa el analizador de clustering.

        Args:
            config: Configuracion de Fase 2C
            logger: Logger para registro
        """
        self.config = config
        self.logger = logger

        self.kmeans = None
        self.features_clustering = None
        self.labels = None

    def preparar_features_fotografias(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Prepara features por fotografia para clustering.

        Features utilizadas:
        - IoU medio de todos los modelos
        - Desviacion estandar del IoU entre modelos
        - IoU maximo alcanzado
        - IoU minimo alcanzado
        - Numero de modelos con IoU > 0.8

        Args:
            df: DataFrame fusionado

        Returns:
            DataFrame con features por fotografia
        """
        self.logger.info("Preparando features de fotografias para clustering...")

        # Agregar por fotografia
        features_foto = df.groupby('codigo_foto').agg({
            'iou': ['mean', 'std', 'min', 'max', 'count']
        }).round(4)

        features_foto.columns = ['iou_mean', 'iou_std', 'iou_min', 'iou_max', 'n_evaluaciones']
        features_foto = features_foto.reset_index()

        # Calcular numero de modelos con IoU > 0.8
        modelos_buenos = df[df['iou'] > 0.8].groupby('codigo_foto')['modelo'].nunique()
        modelos_buenos = modelos_buenos.reset_index()
        modelos_buenos.columns = ['codigo_foto', 'n_modelos_iou_alto']

        # Combinar
        features_foto = features_foto.merge(modelos_buenos, on='codigo_foto', how='left')
        features_foto['n_modelos_iou_alto'].fillna(0, inplace=True)

        # Calcular rango de IoU
        features_foto['iou_rango'] = features_foto['iou_max'] - features_foto['iou_min']

        self.logger.info(f"  Fotografias: {len(features_foto)}")
        self.logger.info(f"  Features: {features_foto.columns.tolist()}")

        return features_foto

    def ejecutar_kmeans(self, df_features: pd.DataFrame) -> Dict[str, Any]:
        """
        Ejecuta clustering K-means sobre fotografias.

        Args:
            df_features: DataFrame con features por fotografia

        Returns:
            Diccionario con resultados del clustering
        """
        self.logger.info(f"Ejecutando K-means con k={self.config.n_clusters}...")

        # Seleccionar features numericas para clustering
        cols_features = ['iou_mean', 'iou_std', 'iou_rango', 'n_modelos_iou_alto']
        self.features_clustering = df_features[cols_features].values

        # Estandarizar
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(self.features_clustering)

        # K-means
        self.kmeans = KMeans(
            n_clusters=self.config.n_clusters,
            random_state=self.config.random_state,
            n_init=10
        )
        self.labels = self.kmeans.fit_predict(X_scaled)

        # Metricas de calidad del clustering
        silhouette = silhouette_score(X_scaled, self.labels)
        calinski = calinski_harabasz_score(X_scaled, self.labels)

        self.logger.info(f"  Silhouette Score: {silhouette:.3f}")
        self.logger.info(f"  Calinski-Harabasz Index: {calinski:.1f}")

        # Reordenar clusters por IoU medio (0=facil, 2=dificil)
        df_temp = df_features.copy()
        df_temp['cluster_original'] = self.labels

        medias_cluster = df_temp.groupby('cluster_original')['iou_mean'].mean()
        orden_clusters = medias_cluster.sort_values(ascending=False).index.tolist()

        mapeo_clusters = {old: new for new, old in enumerate(orden_clusters)}
        self.labels = np.array([mapeo_clusters[l] for l in self.labels])

        self.logger.info("  Clusters reordenados por IoU medio (0=Facil, 2=Dificil)")

        # Caracterizar clusters
        df_temp['cluster'] = self.labels
        caracterizacion = {}

        for cluster_id in range(self.config.n_clusters):
            df_c = df_temp[df_temp['cluster'] == cluster_id]
            caracterizacion[cluster_id] = {
                'nombre': NOMBRES_CLUSTERS.get(cluster_id, f'Cluster {cluster_id}'),
                'n_fotos': int(len(df_c)),
                'iou_mean': float(df_c['iou_mean'].mean()),
                'iou_std_mean': float(df_c['iou_std'].mean()),
                'fotos': df_c['codigo_foto'].tolist()
            }

            self.logger.info(
                f"    Cluster {cluster_id} ({NOMBRES_CLUSTERS.get(cluster_id, '')}): "
                f"{len(df_c)} fotos, IoU medio={df_c['iou_mean'].mean():.3f}"
            )

        resultados = {
            'n_clusters': self.config.n_clusters,
            'silhouette_score': float(silhouette),
            'calinski_harabasz': float(calinski),
            'caracterizacion': caracterizacion
        }

        return resultados

    def ejecutar_clustering_jerarquico(self, df_features: pd.DataFrame) -> np.ndarray:
        """
        Ejecuta clustering jerarquico para generar dendrograma.

        Args:
            df_features: DataFrame con features por fotografia

        Returns:
            Matriz de linkage para dendrograma
        """
        self.logger.info("Ejecutando clustering jerarquico...")

        cols_features = ['iou_mean', 'iou_std', 'iou_rango']
        X = df_features[cols_features].values

        # Estandarizar
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        # Linkage jerarquico (Ward)
        linkage_matrix = linkage(X_scaled, method='ward')

        return linkage_matrix

    def obtener_dataframe_clusters(self, df_features: pd.DataFrame) -> pd.DataFrame:
        """
        Genera DataFrame con asignacion de clusters.

        Args:
            df_features: DataFrame con features por fotografia

        Returns:
            DataFrame con codigo_foto, cluster, nombre_cluster
        """
        df_clusters = df_features[['codigo_foto']].copy()
        df_clusters['cluster'] = self.labels
        df_clusters['nombre_cluster'] = df_clusters['cluster'].map(NOMBRES_CLUSTERS)

        # Anadir features originales
        for col in ['iou_mean', 'iou_std', 'iou_rango']:
            if col in df_features.columns:
                df_clusters[col] = df_features[col].values

        return df_clusters


# =============================================================================
# CLASE: ANALIZADOR DE CORRELACIONES
# =============================================================================

class AnalizadorCorrelacionesMetricas:
    """
    Analiza correlaciones entre metricas para identificar redundancias.
    """

    def __init__(self, config: ConfiguracionFase2C, logger: logging.Logger):
        """
        Inicializa el analizador de correlaciones.

        Args:
            config: Configuracion de Fase 2C
            logger: Logger para registro
        """
        self.config = config
        self.logger = logger

        self.matriz_correlacion = None

    def calcular_matriz_correlacion(self, df_metricas: pd.DataFrame) -> pd.DataFrame:
        """
        Calcula matriz de correlacion de Pearson entre metricas.

        Args:
            df_metricas: DataFrame con metricas

        Returns:
            Matriz de correlacion
        """
        self.logger.info("Calculando matriz de correlacion entre metricas...")

        self.matriz_correlacion = df_metricas.corr(method='pearson')

        self.logger.info(f"  Dimensiones: {self.matriz_correlacion.shape}")

        return self.matriz_correlacion

    def identificar_metricas_redundantes(self) -> pd.DataFrame:
        """
        Identifica pares de metricas con correlacion |r| > umbral.

        Returns:
            DataFrame con pares redundantes
        """
        self.logger.info(
            f"Identificando metricas redundantes (|r| > {self.config.umbral_redundancia})..."
        )

        if self.matriz_correlacion is None:
            raise ValueError("Ejecute calcular_matriz_correlacion primero")

        redundantes = []

        # Recorrer triangulo superior de la matriz
        n = len(self.matriz_correlacion)
        for i in range(n):
            for j in range(i + 1, n):
                r = self.matriz_correlacion.iloc[i, j]

                if abs(r) > self.config.umbral_redundancia:
                    redundantes.append({
                        'metrica_1': self.matriz_correlacion.index[i],
                        'metrica_2': self.matriz_correlacion.columns[j],
                        'correlacion': float(r),
                        'correlacion_abs': float(abs(r))
                    })

        df_redundantes = pd.DataFrame(redundantes)

        if len(df_redundantes) > 0:
            df_redundantes = df_redundantes.sort_values('correlacion_abs', ascending=False)
            self.logger.info(f"  Pares redundantes encontrados: {len(df_redundantes)}")
        else:
            self.logger.info("  No se encontraron metricas redundantes")

        return df_redundantes

    def sugerir_metricas_a_eliminar(self, df_redundantes: pd.DataFrame) -> List[str]:
        """
        Sugiere metricas a eliminar para reducir redundancia.

        Criterio: Eliminar la metrica menos interpretable del par.

        Args:
            df_redundantes: DataFrame con pares redundantes

        Returns:
            Lista de metricas sugeridas para eliminacion
        """
        # Metricas prioritarias a mantener (mas interpretables)
        prioridad = ['iou', 'dice', 'precision', 'recall', 'boundary_iou']

        a_eliminar = set()

        for _, row in df_redundantes.iterrows():
            m1, m2 = row['metrica_1'], row['metrica_2']

            # Si una esta en prioridad y otra no, eliminar la no prioritaria
            m1_prioritaria = m1 in prioridad
            m2_prioritaria = m2 in prioridad

            if m1_prioritaria and not m2_prioritaria:
                a_eliminar.add(m2)
            elif m2_prioritaria and not m1_prioritaria:
                a_eliminar.add(m1)
            else:
                # Si ambas o ninguna es prioritaria, eliminar la segunda
                a_eliminar.add(m2)

        return list(a_eliminar)


# =============================================================================
# CLASE: GENERADOR DE VISUALIZACIONES
# =============================================================================

class GeneradorVisualizaciones:
    """
    Genera visualizaciones para Fase 2C.
    """

    def __init__(self, config: ConfiguracionFase2C, logger: logging.Logger):
        """
        Inicializa el generador de visualizaciones.

        Args:
            config: Configuracion de Fase 2C
            logger: Logger para registro
        """
        self.config = config
        self.logger = logger

        self.ruta_visualizaciones = config.ruta_salida / "visualizaciones"

    def _crear_directorio(self):
        """Crea el directorio de visualizaciones si no existe."""
        self.ruta_visualizaciones.mkdir(parents=True, exist_ok=True)

    def generar_varianza_explicada(self, df_varianza: pd.DataFrame) -> Path:
        """
        Genera grafico de varianza explicada por componente (scree plot).

        Args:
            df_varianza: DataFrame con varianza por componente

        Returns:
            Ruta al archivo generado
        """
        self._crear_directorio()
        self.logger.info("Generando grafico de varianza explicada...")

        fig, ax = plt.subplots(figsize=(10, 6))

        x = range(1, len(df_varianza) + 1)

        # Barras de varianza individual
        bars = ax.bar(x, df_varianza['varianza_explicada'] * 100,
                     color='steelblue', alpha=0.7, label='Varianza individual')

        # Linea de varianza acumulada
        ax.plot(x, df_varianza['varianza_acumulada'] * 100,
               'ro-', linewidth=2, markersize=8, label='Varianza acumulada')

        # Linea de referencia 80%
        ax.axhline(y=80, color='gray', linestyle='--', alpha=0.5, label='80% varianza')

        # Etiquetas en barras
        for bar, val in zip(bars, df_varianza['varianza_explicada']):
            ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                   f'{val*100:.1f}%', ha='center', va='bottom', fontsize=9)

        ax.set_xlabel('Componente Principal')
        ax.set_ylabel('Varianza Explicada (%)')
        ax.set_title('PCA: Varianza Explicada por Componente', fontweight='bold')
        ax.set_xticks(x)
        ax.set_xticklabels([f'PC{i}' for i in x])
        ax.legend(loc='center right')
        ax.set_ylim(0, 105)

        plt.tight_layout()

        ruta = self.ruta_visualizaciones / 'pca_varianza_explicada.png'
        fig.savefig(ruta, dpi=300, bbox_inches='tight')
        plt.close(fig)

        return ruta

    def generar_biplot(self, analizador_pca: AnalizadorPCA,
                       df_meta: pd.DataFrame) -> Path:
        """
        Genera biplot de PCA (scores + loadings).

        Args:
            analizador_pca: Analizador PCA ejecutado
            df_meta: DataFrame con metadata (modelo)

        Returns:
            Ruta al archivo generado
        """
        self._crear_directorio()
        self.logger.info("Generando biplot PCA...")

        fig, ax = plt.subplots(figsize=(12, 10))

        # Scores (puntos)
        scores = analizador_pca.X_pca[:, :2]
        modelos = df_meta['modelo'].values

        for modelo in np.unique(modelos):
            mask = modelos == modelo
            color = COLORES_MODELOS.get(modelo, '#666666')
            ax.scatter(scores[mask, 0], scores[mask, 1],
                      c=color, label=modelo, alpha=0.6, s=30)

        # Loadings (flechas)
        loadings = analizador_pca.pca.components_[:2, :].T
        feature_names = analizador_pca.feature_names

        # Escalar loadings para visualizacion
        scale = np.abs(scores).max() / np.abs(loadings).max() * 0.8

        for i, (name, loading) in enumerate(zip(feature_names, loadings)):
            ax.arrow(0, 0, loading[0] * scale, loading[1] * scale,
                    head_width=0.15, head_length=0.1, fc='red', ec='red', alpha=0.7)

            # Etiqueta
            ax.text(loading[0] * scale * 1.15, loading[1] * scale * 1.15,
                   name, fontsize=8, ha='center', va='center', color='darkred')

        ax.axhline(y=0, color='gray', linestyle='-', linewidth=0.5)
        ax.axvline(x=0, color='gray', linestyle='-', linewidth=0.5)

        ax.set_xlabel(f'PC1 ({analizador_pca.pca.explained_variance_ratio_[0]*100:.1f}%)')
        ax.set_ylabel(f'PC2 ({analizador_pca.pca.explained_variance_ratio_[1]*100:.1f}%)')
        ax.set_title('PCA Biplot: Evaluaciones de Segmentacion', fontweight='bold')
        ax.legend(loc='upper right', title='Modelo')

        plt.tight_layout()

        ruta = self.ruta_visualizaciones / 'pca_biplot.png'
        fig.savefig(ruta, dpi=300, bbox_inches='tight')
        plt.close(fig)

        return ruta

    def generar_scatter_pca_clusters(self, df_scores: pd.DataFrame,
                                      df_clusters: pd.DataFrame) -> Path:
        """
        Genera scatter plot de PC1 vs PC2 coloreado por cluster de foto.

        Args:
            df_scores: DataFrame con scores PCA
            df_clusters: DataFrame con asignacion de clusters

        Returns:
            Ruta al archivo generado
        """
        self._crear_directorio()
        self.logger.info("Generando scatter PCA con clusters...")

        # Merge scores con clusters
        df_plot = df_scores.merge(
            df_clusters[['codigo_foto', 'cluster', 'nombre_cluster']],
            on='codigo_foto',
            how='left'
        )

        fig, ax = plt.subplots(figsize=(12, 8))

        for cluster_id in sorted(df_plot['cluster'].unique()):
            mask = df_plot['cluster'] == cluster_id
            nombre = NOMBRES_CLUSTERS.get(cluster_id, f'Cluster {cluster_id}')
            color = COLORES_CLUSTERS.get(cluster_id, '#666666')

            ax.scatter(
                df_plot.loc[mask, 'PC1'],
                df_plot.loc[mask, 'PC2'],
                c=color, label=f'{nombre}', alpha=0.6, s=40, edgecolors='white'
            )

        ax.axhline(y=0, color='gray', linestyle='-', linewidth=0.5)
        ax.axvline(x=0, color='gray', linestyle='-', linewidth=0.5)

        ax.set_xlabel('PC1')
        ax.set_ylabel('PC2')
        ax.set_title('PCA: Evaluaciones por Nivel de Dificultad de Fotografia', fontweight='bold')
        ax.legend(title='Dificultad')

        plt.tight_layout()

        ruta = self.ruta_visualizaciones / 'pca_scatter_clusters.png'
        fig.savefig(ruta, dpi=300, bbox_inches='tight')
        plt.close(fig)

        return ruta

    def generar_dendrograma(self, linkage_matrix: np.ndarray,
                            codigos_foto: List[str]) -> Path:
        """
        Genera dendrograma del clustering jerarquico.

        Args:
            linkage_matrix: Matriz de linkage
            codigos_foto: Lista de codigos de fotografia

        Returns:
            Ruta al archivo generado
        """
        self._crear_directorio()
        self.logger.info("Generando dendrograma...")

        fig, ax = plt.subplots(figsize=(14, 8))

        # Simplificar etiquetas
        etiquetas = [c.replace('_DSC', '').replace('_', '') for c in codigos_foto]

        dendrogram(
            linkage_matrix,
            labels=etiquetas,
            leaf_rotation=90,
            leaf_font_size=9,
            ax=ax,
            color_threshold=0.7 * max(linkage_matrix[:, 2])
        )

        ax.set_xlabel('Fotografia')
        ax.set_ylabel('Distancia (Ward)')
        ax.set_title('Dendrograma: Clustering Jerarquico de Fotografias por Dificultad',
                    fontweight='bold')

        plt.tight_layout()

        ruta = self.ruta_visualizaciones / 'dendrograma_fotografias.png'
        fig.savefig(ruta, dpi=300, bbox_inches='tight')
        plt.close(fig)

        return ruta

    def generar_heatmap_correlaciones(self, matriz_corr: pd.DataFrame) -> Path:
        """
        Genera heatmap de la matriz de correlaciones.

        Args:
            matriz_corr: Matriz de correlacion

        Returns:
            Ruta al archivo generado
        """
        self._crear_directorio()
        self.logger.info("Generando heatmap de correlaciones...")

        fig, ax = plt.subplots(figsize=(14, 12))

        # Mascara para triangulo superior
        mask = np.triu(np.ones_like(matriz_corr, dtype=bool), k=1)

        sns.heatmap(
            matriz_corr,
            mask=mask,
            annot=True,
            fmt='.2f',
            cmap='RdBu_r',
            center=0,
            vmin=-1,
            vmax=1,
            square=True,
            linewidths=0.5,
            cbar_kws={'shrink': 0.8, 'label': 'Correlacion de Pearson'},
            ax=ax,
            annot_kws={'size': 8}
        )

        ax.set_title('Matriz de Correlaciones entre Metricas de Segmentacion',
                    fontweight='bold', pad=20)

        plt.tight_layout()

        ruta = self.ruta_visualizaciones / 'heatmap_correlaciones.png'
        fig.savefig(ruta, dpi=300, bbox_inches='tight')
        plt.close(fig)

        return ruta

    def generar_caracterizacion_clusters(self, df_features: pd.DataFrame,
                                          labels: np.ndarray) -> Path:
        """
        Genera visualizacion de caracterizacion de clusters.

        Args:
            df_features: DataFrame con features por fotografia
            labels: Array con etiquetas de cluster

        Returns:
            Ruta al archivo generado
        """
        self._crear_directorio()
        self.logger.info("Generando caracterizacion de clusters...")

        df_plot = df_features.copy()
        df_plot['cluster'] = labels
        df_plot['nombre_cluster'] = df_plot['cluster'].map(NOMBRES_CLUSTERS)

        fig, axes = plt.subplots(2, 2, figsize=(14, 12))

        # 1. Box plot IoU medio por cluster
        ax = axes[0, 0]
        colores = [COLORES_CLUSTERS[i] for i in sorted(df_plot['cluster'].unique())]

        df_plot.boxplot(column='iou_mean', by='nombre_cluster', ax=ax,
                       patch_artist=True)

        for patch, color in zip(ax.patches, colores):
            patch.set_facecolor(color)
            patch.set_alpha(0.7)

        ax.set_title('IoU Medio por Cluster')
        ax.set_xlabel('Cluster')
        ax.set_ylabel('IoU Medio')
        plt.suptitle('')

        # 2. Box plot variabilidad por cluster
        ax = axes[0, 1]
        df_plot.boxplot(column='iou_std', by='nombre_cluster', ax=ax,
                       patch_artist=True)

        for patch, color in zip(ax.patches, colores):
            patch.set_facecolor(color)
            patch.set_alpha(0.7)

        ax.set_title('Variabilidad IoU por Cluster')
        ax.set_xlabel('Cluster')
        ax.set_ylabel('Desviacion Estandar IoU')
        plt.suptitle('')

        # 3. Scatter IoU medio vs variabilidad
        ax = axes[1, 0]
        for cluster_id in sorted(df_plot['cluster'].unique()):
            mask = df_plot['cluster'] == cluster_id
            ax.scatter(
                df_plot.loc[mask, 'iou_mean'],
                df_plot.loc[mask, 'iou_std'],
                c=COLORES_CLUSTERS[cluster_id],
                label=NOMBRES_CLUSTERS[cluster_id],
                s=80, alpha=0.7, edgecolors='white'
            )

            # Etiquetas de fotos
            for _, row in df_plot[mask].iterrows():
                ax.annotate(
                    row['codigo_foto'].replace('_DSC', ''),
                    (row['iou_mean'], row['iou_std']),
                    fontsize=7, alpha=0.7
                )

        ax.set_xlabel('IoU Medio')
        ax.set_ylabel('Variabilidad IoU (std)')
        ax.set_title('Relacion IoU Medio vs Variabilidad')
        ax.legend()

        # 4. Barras de conteo por cluster
        ax = axes[1, 1]
        conteos = df_plot.groupby('nombre_cluster').size()
        conteos = conteos.reindex(['Facil', 'Medio', 'Dificil'])

        bars = ax.bar(conteos.index, conteos.values,
                     color=[COLORES_CLUSTERS[i] for i in range(len(conteos))],
                     alpha=0.7, edgecolor='black')

        for bar, val in zip(bars, conteos.values):
            ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.2,
                   str(val), ha='center', va='bottom', fontweight='bold')

        ax.set_xlabel('Cluster')
        ax.set_ylabel('Numero de Fotografias')
        ax.set_title('Distribucion de Fotografias por Cluster')

        fig.suptitle('Caracterizacion de Clusters de Dificultad',
                    fontsize=14, fontweight='bold', y=1.02)

        plt.tight_layout()

        ruta = self.ruta_visualizaciones / 'clusters_caracterizacion.png'
        fig.savefig(ruta, dpi=300, bbox_inches='tight')
        plt.close(fig)

        return ruta


# =============================================================================
# CLASE: GENERADOR DE REPORTES
# =============================================================================

class GeneradorReporte:
    """
    Genera reporte markdown con resultados de Fase 2C.
    """

    def __init__(self, config: ConfiguracionFase2C, logger: logging.Logger):
        """
        Inicializa el generador de reportes.

        Args:
            config: Configuracion de Fase 2C
            logger: Logger para registro
        """
        self.config = config
        self.logger = logger

    def generar_reporte(self, resultados_pca: Dict,
                        resultados_clustering: Dict,
                        df_redundantes: pd.DataFrame,
                        rutas_visualizaciones: Dict[str, Path]) -> Path:
        """
        Genera reporte markdown completo.

        Args:
            resultados_pca: Resultados del analisis PCA
            resultados_clustering: Resultados del clustering
            df_redundantes: DataFrame con metricas redundantes
            rutas_visualizaciones: Diccionario con rutas a visualizaciones

        Returns:
            Ruta al reporte generado
        """
        self.logger.info("Generando reporte markdown...")

        lineas = [
            "# REPORTE FASE 2C - PCA Y CLUSTERING",
            "",
            f"**Fecha de generacion:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
            "",
            "---",
            "",
            "## 1. Resumen Ejecutivo",
            "",
            "Este reporte presenta los resultados del analisis exploratorio multivariante",
            "sobre las metricas de segmentacion, incluyendo reduccion de dimensionalidad (PCA),",
            "clustering de fotografias por dificultad, y analisis de redundancia entre metricas.",
            "",
            "---",
            "",
            "## 2. Analisis de Componentes Principales (PCA)",
            "",
            f"**Componentes retenidos:** {resultados_pca['n_componentes']}",
            "",
            "### Varianza Explicada",
            "",
            "| Componente | Varianza (%) | Acumulada (%) |",
            "|------------|--------------|---------------|",
        ]

        for i, (var, acum) in enumerate(zip(
            resultados_pca['varianza_explicada'],
            resultados_pca['varianza_acumulada']
        )):
            lineas.append(f"| PC{i+1} | {var*100:.2f} | {acum*100:.2f} |")

        lineas.extend([
            "",
            f"![Varianza Explicada](visualizaciones/pca_varianza_explicada.png)",
            "",
            "### Interpretacion",
            "",
            f"Los primeros {resultados_pca['n_componentes']} componentes explican el "
            f"{resultados_pca['varianza_acumulada'][-1]*100:.1f}% de la varianza total.",
            "",
            "---",
            "",
            "## 3. Clustering de Fotografias",
            "",
            f"**Metodo:** K-means con k={resultados_clustering['n_clusters']}",
            "",
            f"**Silhouette Score:** {resultados_clustering['silhouette_score']:.3f}",
            "",
            f"**Calinski-Harabasz Index:** {resultados_clustering['calinski_harabasz']:.1f}",
            "",
            "### Caracterizacion de Clusters",
            "",
            "| Cluster | Nombre | N Fotos | IoU Medio |",
            "|---------|--------|---------|-----------|",
        ])

        for cluster_id, info in resultados_clustering['caracterizacion'].items():
            lineas.append(
                f"| {cluster_id} | {info['nombre']} | {info['n_fotos']} | "
                f"{info['iou_mean']:.4f} |"
            )

        lineas.extend([
            "",
            f"![Caracterizacion Clusters](visualizaciones/clusters_caracterizacion.png)",
            "",
            "### Fotografias por Cluster",
            "",
        ])

        for cluster_id, info in resultados_clustering['caracterizacion'].items():
            fotos = ", ".join(info['fotos'][:5])
            if len(info['fotos']) > 5:
                fotos += f" ... (+{len(info['fotos'])-5} mas)"
            lineas.append(f"**{info['nombre']}:** {fotos}")
            lineas.append("")

        lineas.extend([
            "---",
            "",
            "## 4. Analisis de Redundancia",
            "",
            f"**Umbral de redundancia:** |r| > {self.config.umbral_redundancia}",
            "",
        ])

        if len(df_redundantes) > 0:
            lineas.extend([
                f"**Pares redundantes encontrados:** {len(df_redundantes)}",
                "",
                "| Metrica 1 | Metrica 2 | Correlacion |",
                "|-----------|-----------|-------------|",
            ])

            for _, row in df_redundantes.head(10).iterrows():
                lineas.append(
                    f"| {row['metrica_1']} | {row['metrica_2']} | {row['correlacion']:.3f} |"
                )

            if len(df_redundantes) > 10:
                lineas.append(f"| ... | ... | ... |")
                lineas.append(f"| (Total: {len(df_redundantes)} pares) | | |")
        else:
            lineas.append("No se encontraron metricas redundantes con el umbral especificado.")

        lineas.extend([
            "",
            f"![Heatmap Correlaciones](visualizaciones/heatmap_correlaciones.png)",
            "",
            "---",
            "",
            "## 5. Archivos Generados",
            "",
            "| Archivo | Descripcion |",
            "|---------|-------------|",
            "| pca_componentes.csv | Scores PC1-PCn por evaluacion |",
            "| pca_varianza_explicada.csv | Varianza por componente |",
            "| pca_loadings.csv | Pesos de metricas en cada PC |",
            "| clusters_fotografias.csv | Asignacion de cluster por foto |",
            "| correlaciones_metricas.csv | Matriz de correlaciones |",
            "| metricas_redundantes.csv | Pares con alta correlacion |",
            "",
            "---",
            "",
            "*Reporte generado automaticamente por Fase 2C*",
        ])

        contenido = "\n".join(lineas)

        ruta = self.config.ruta_salida / "REPORTE_FASE2C.md"
        with open(ruta, 'w', encoding='utf-8') as f:
            f.write(contenido)

        return ruta


# =============================================================================
# CLASE: ORQUESTADOR FASE 2C
# =============================================================================

class OrquestadorFase2C:
    """
    Orquesta la ejecucion completa de Fase 2C.
    """

    def __init__(self, config: ConfiguracionFase2C):
        """
        Inicializa el orquestador.

        Args:
            config: Configuracion de Fase 2C
        """
        self.config = config
        self.logger = self._configurar_logger()

        # Componentes
        self.cargador = CargadorDatosFase2C(config, self.logger)
        self.analizador_pca = AnalizadorPCA(config, self.logger)
        self.analizador_clustering = AnalizadorClustering(config, self.logger)
        self.analizador_correlaciones = AnalizadorCorrelacionesMetricas(config, self.logger)
        self.generador_viz = GeneradorVisualizaciones(config, self.logger)
        self.generador_reporte = GeneradorReporte(config, self.logger)

        # Resultados
        self.df_original = None
        self.df_metricas = None
        self.df_meta = None
        self.metricas_disponibles = None
        self.resultados_pca = None
        self.resultados_clustering = None
        self.df_redundantes = None
        self.rutas_visualizaciones = {}

    def _configurar_logger(self) -> logging.Logger:
        """Configura el logger para Fase 2C."""
        logger = logging.getLogger('Fase2C')
        logger.setLevel(logging.INFO)

        # Limpiar handlers existentes
        logger.handlers = []

        handler = logging.StreamHandler(sys.stdout)
        handler.setLevel(logging.INFO)
        formatter = logging.Formatter(
            '[%(asctime)s] %(levelname)-8s | %(message)s',
            datefmt='%H:%M:%S'
        )
        handler.setFormatter(formatter)
        logger.addHandler(handler)

        return logger

    def ejecutar(self) -> None:
        """
        Ejecuta el pipeline completo de Fase 2C.
        """
        self.logger.info("=" * 70)
        self.logger.info("FASE 2C - PCA, CLUSTERING Y ANALISIS DE REDUNDANCIA")
        self.logger.info("=" * 70)

        inicio = datetime.now()

        # Paso 1: Cargar datos
        self.logger.info("\n[PASO 1/8] Cargando datos de Fase 2B...")
        self.df_original = self.cargador.cargar_datos_fusionados()

        # Paso 2: Identificar metricas disponibles
        self.logger.info("\n[PASO 2/8] Identificando metricas disponibles...")
        self.metricas_disponibles = self.cargador.identificar_metricas_disponibles(
            self.df_original
        )

        if len(self.metricas_disponibles) < 3:
            raise ValueError(
                f"Insuficientes metricas para PCA: {len(self.metricas_disponibles)}"
            )

        # Paso 3: Preparar matriz de metricas
        self.logger.info("\n[PASO 3/8] Preparando matriz de metricas...")
        self.df_metricas, self.df_meta = self.cargador.preparar_matriz_metricas(
            self.df_original, self.metricas_disponibles
        )

        # Paso 4: Ejecutar PCA
        self.logger.info("\n[PASO 4/8] Ejecutando PCA...")
        self.resultados_pca = self.analizador_pca.ejecutar_pca(self.df_metricas)

        # Paso 5: Ejecutar clustering
        self.logger.info("\n[PASO 5/8] Ejecutando clustering de fotografias...")
        df_features_foto = self.analizador_clustering.preparar_features_fotografias(
            self.df_original
        )
        self.resultados_clustering = self.analizador_clustering.ejecutar_kmeans(df_features_foto)
        linkage_matrix = self.analizador_clustering.ejecutar_clustering_jerarquico(df_features_foto)

        # Paso 6: Analizar correlaciones
        self.logger.info("\n[PASO 6/8] Analizando correlaciones entre metricas...")
        matriz_corr = self.analizador_correlaciones.calcular_matriz_correlacion(self.df_metricas)
        self.df_redundantes = self.analizador_correlaciones.identificar_metricas_redundantes()

        # Paso 7: Generar visualizaciones
        self.logger.info("\n[PASO 7/8] Generando visualizaciones...")

        df_varianza = self.analizador_pca.obtener_dataframe_varianza()
        self.rutas_visualizaciones['varianza'] = self.generador_viz.generar_varianza_explicada(
            df_varianza
        )

        self.rutas_visualizaciones['biplot'] = self.generador_viz.generar_biplot(
            self.analizador_pca, self.df_meta
        )

        df_scores = self.analizador_pca.obtener_dataframe_scores(self.df_meta)
        df_clusters = self.analizador_clustering.obtener_dataframe_clusters(df_features_foto)

        self.rutas_visualizaciones['scatter_clusters'] = self.generador_viz.generar_scatter_pca_clusters(
            df_scores, df_clusters
        )

        self.rutas_visualizaciones['dendrograma'] = self.generador_viz.generar_dendrograma(
            linkage_matrix, df_features_foto['codigo_foto'].tolist()
        )

        self.rutas_visualizaciones['heatmap'] = self.generador_viz.generar_heatmap_correlaciones(
            matriz_corr
        )

        self.rutas_visualizaciones['caracterizacion'] = self.generador_viz.generar_caracterizacion_clusters(
            df_features_foto, self.analizador_clustering.labels
        )

        # Paso 8: Guardar resultados
        self.logger.info("\n[PASO 8/8] Guardando resultados...")
        self._guardar_resultados(df_scores, df_clusters, matriz_corr, df_features_foto)

        # Generar reporte
        self.generador_reporte.generar_reporte(
            self.resultados_pca,
            self.resultados_clustering,
            self.df_redundantes,
            self.rutas_visualizaciones
        )

        duracion = (datetime.now() - inicio).total_seconds()

        self.logger.info("\n" + "=" * 70)
        self.logger.info("FASE 2C COMPLETADA")
        self.logger.info("=" * 70)
        self.logger.info(f"Duracion: {duracion:.1f} segundos")
        self.logger.info(f"Metricas analizadas: {len(self.metricas_disponibles)}")
        self.logger.info(f"Componentes PCA: {self.resultados_pca['n_componentes']}")
        self.logger.info(f"Clusters identificados: {self.config.n_clusters}")
        self.logger.info(f"Pares redundantes: {len(self.df_redundantes)}")
        self.logger.info(f"Resultados en: {self.config.ruta_salida}")

    def _guardar_resultados(self, df_scores: pd.DataFrame,
                            df_clusters: pd.DataFrame,
                            matriz_corr: pd.DataFrame,
                            df_features_foto: pd.DataFrame) -> None:
        """
        Guarda todos los resultados en archivos.

        Args:
            df_scores: DataFrame con scores PCA
            df_clusters: DataFrame con clusters
            matriz_corr: Matriz de correlacion
            df_features_foto: Features de fotografias
        """
        # Crear directorio de salida
        self.config.ruta_salida.mkdir(parents=True, exist_ok=True)

        # Scores PCA
        ruta = self.config.ruta_salida / 'pca_componentes.csv'
        df_scores.to_csv(ruta, index=False)
        self.logger.info(f"  Guardado: {ruta}")

        # Varianza explicada
        ruta = self.config.ruta_salida / 'pca_varianza_explicada.csv'
        self.analizador_pca.obtener_dataframe_varianza().to_csv(ruta, index=False)
        self.logger.info(f"  Guardado: {ruta}")

        # Loadings
        ruta = self.config.ruta_salida / 'pca_loadings.csv'
        self.analizador_pca.obtener_dataframe_loadings().to_csv(ruta)
        self.logger.info(f"  Guardado: {ruta}")

        # Clusters
        ruta = self.config.ruta_salida / 'clusters_fotografias.csv'
        df_clusters.to_csv(ruta, index=False)
        self.logger.info(f"  Guardado: {ruta}")

        # Features de fotografias (con cluster)
        df_features_foto['cluster'] = self.analizador_clustering.labels
        df_features_foto['nombre_cluster'] = df_features_foto['cluster'].map(NOMBRES_CLUSTERS)
        ruta = self.config.ruta_salida / 'features_fotografias.csv'
        df_features_foto.to_csv(ruta, index=False)
        self.logger.info(f"  Guardado: {ruta}")

        # Matriz de correlaciones
        ruta = self.config.ruta_salida / 'correlaciones_metricas.csv'
        matriz_corr.to_csv(ruta)
        self.logger.info(f"  Guardado: {ruta}")

        # Metricas redundantes
        ruta = self.config.ruta_salida / 'metricas_redundantes.csv'
        self.df_redundantes.to_csv(ruta, index=False)
        self.logger.info(f"  Guardado: {ruta}")

        # Resumen JSON
        resumen = {
            'timestamp': datetime.now().isoformat(),
            'metricas_analizadas': self.metricas_disponibles,
            'pca': {
                'n_componentes': self.resultados_pca['n_componentes'],
                'varianza_total': self.resultados_pca['varianza_acumulada'][-1]
            },
            'clustering': {
                'n_clusters': self.config.n_clusters,
                'silhouette': self.resultados_clustering['silhouette_score'],
                'caracterizacion': self.resultados_clustering['caracterizacion']
            },
            'redundancia': {
                'umbral': self.config.umbral_redundancia,
                'pares_encontrados': len(self.df_redundantes)
            }
        }

        ruta = self.config.ruta_salida / 'resumen_fase2c.json'
        with open(ruta, 'w', encoding='utf-8') as f:
            json.dump(resumen, f, indent=2, ensure_ascii=False, default=str)
        self.logger.info(f"  Guardado: {ruta}")

    def imprimir_resumen(self) -> None:
        """Imprime resumen de resultados."""
        print("\n" + "=" * 70)
        print("RESUMEN DE RESULTADOS FASE 2C")
        print("=" * 70)

        # PCA
        print("\nANALISIS DE COMPONENTES PRINCIPALES:")
        print("-" * 50)
        print(f"  Metricas analizadas: {len(self.metricas_disponibles)}")
        print(f"  Componentes retenidos: {self.resultados_pca['n_componentes']}")
        print(f"  Varianza explicada: {self.resultados_pca['varianza_acumulada'][-1]*100:.1f}%")

        # Clustering
        print("\nCLUSTERING DE FOTOGRAFIAS:")
        print("-" * 50)
        print(f"  Silhouette Score: {self.resultados_clustering['silhouette_score']:.3f}")

        for cluster_id, info in self.resultados_clustering['caracterizacion'].items():
            print(f"  {info['nombre']}: {info['n_fotos']} fotos (IoU medio: {info['iou_mean']:.4f})")

        # Redundancia
        print("\nANALISIS DE REDUNDANCIA:")
        print("-" * 50)
        print(f"  Umbral: |r| > {self.config.umbral_redundancia}")
        print(f"  Pares redundantes: {len(self.df_redundantes)}")

        if len(self.df_redundantes) > 0:
            print("  Top 5 pares mas correlacionados:")
            for _, row in self.df_redundantes.head(5).iterrows():
                print(f"    {row['metrica_1']} <-> {row['metrica_2']}: r={row['correlacion']:.3f}")


# =============================================================================
# FUNCION PRINCIPAL
# =============================================================================

def ejecutar_fase2c(ruta_base_tfm: str,
                    ruta_datos_fase2b: str = None,
                    n_clusters: int = 3) -> OrquestadorFase2C:
    """
    Ejecuta el pipeline completo de Fase 2C.

    Args:
        ruta_base_tfm: Ruta base del proyecto TFM
        ruta_datos_fase2b: Ruta al CSV fusionado (opcional)
        n_clusters: Numero de clusters para K-means

    Returns:
        OrquestadorFase2C con resultados
    """
    config = ConfiguracionFase2C(
        ruta_base_tfm=Path(ruta_base_tfm),
        n_clusters=n_clusters
    )

    if ruta_datos_fase2b:
        config.ruta_datos_fase2b = Path(ruta_datos_fase2b)

    orquestador = OrquestadorFase2C(config)
    orquestador.ejecutar()
    orquestador.imprimir_resumen()

    return orquestador


# =============================================================================
# PUNTO DE ENTRADA
# =============================================================================

if __name__ == "__main__":

    # ==========================================================================
    # CONFIGURACION PARA GOOGLE COLAB
    # ==========================================================================

    from google.colab import drive
    drive.mount('/content/drive')

    # Rutas
    RUTA_BASE_TFM = Path("/content/drive/MyDrive/TFM")
    RUTA_FASE2B = RUTA_BASE_TFM / "3_Analisis" / "fase2b_correlaciones" / "metricas_fusionadas.csv"

    # Verificar archivo de entrada
    if not RUTA_FASE2B.exists():
        print(f"ERROR: No se encuentra {RUTA_FASE2B}")
        print("Ejecute Fase 2B primero.")
    else:
        print(f"Archivo encontrado: {RUTA_FASE2B}")

        # Ejecutar Fase 2C
        orquestador = ejecutar_fase2c(
            ruta_base_tfm=str(RUTA_BASE_TFM),
            ruta_datos_fase2b=str(RUTA_FASE2B),
            n_clusters=3
        )